In [1]:
import os
os.chdir("../")

In [159]:
import glob
import pandas as pd
import time

In [302]:
resultDir = 'results'
problem = 'facilities' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [303]:
eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
eval_file = eval_files[-1]

In [304]:
df = pd.read_csv(eval_file)
df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

In [307]:
# process_time = time.process_time()
# perf_time = time.perf_counter()
df2 = df[df['sampling_strategy'] == 'uniform5']
for i in range(0, len(df)):
    ref_row = df2[
        (df2['policy'] == df.loc[i].policy) &
        (df2['seed'] == df.loc[i].seed) &
        (df2['type'] == df.loc[i].type) &
        (df2['instance'] == df.loc[i].instance)
    ]
    df.at[i,'stime'] /= ref_row.stime
    df.at[i,'nnodes'] = df.at[i,'nnodes'].astype(float) / ref_row.nnodes.astype(float)
    df.at[i,'nlps'] = df.at[i,'nlps'].astype(float) / ref_row.nlps.astype(float)
    df.at[i,'walltime'] = df.at[i,'walltime'].astype(float) / ref_row.walltime.astype(float)
    df.at[i,'proctime'] = df.at[i,'proctime'].astype(float) / ref_row.proctime.astype(float)
 

In [253]:
import numpy as np

In [273]:
df.loc[(df['sampling_strategy'] == 'depthK') & (df['type']=='small'),'stime'].mean()

1.03691385447494

In [308]:
df.groupby(['type','sampling_strategy']).mean()

seed    nnodes      nlps     stime       gap  \
type   sampling_strategy                                                 
medium depthK                2  1.025238  1.023790  1.018855  0.000000   
       depthK2               2  1.062313  1.039587  1.028057  0.000000   
       uniform5              2  1.000000  1.000000  1.000000  0.000000   
small  depthK                2  1.007539  0.995745  0.988374  0.000131   
       depthK2               2  1.015996  1.007330  1.002060  0.000174   
       uniform5              2  1.000000  1.000000  1.000000  0.000191   

                          ndomchgs  ncutoffs  walltime  proctime  
type   sampling_strategy                                          
medium depthK                    0         0  1.018507  1.018637  
       depthK2                   0         0  1.028073  1.028072  
       uniform5                  0         0  1.000000  1.000000  
small  depthK                    0         0  0.988560  0.988305  
       depthK2                   0         0  1.002174  1.002097  
       uniform5                  0         0  1.000000  1.000000

In [288]:
from scipy import stats

In [290]:
stats.mstats.gmean(df.loc[(df['sampling_strategy'] == 'depthK') & (df['type']=='small'),'stime'] + 1) - 1

1.036213421932457

In [291]:
df

,policy,sampling_strategy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:baseline,uniform5,0,small,data/instances/setcover/transfer_500r_1000c_0....,1.000000,1.000000,1.000000,0.0,optimal,0,0,1.000000,1.000000
1,gcnn:baseline,uniform5,1,small,data/instances/setcover/transfer_500r_1000c_0....,1.000000,1.000000,1.000000,0.0,optimal,0,0,1.000000,1.000000
2,gcnn:baseline,uniform5,2,small,data/instances/setcover/transfer_500r_1000c_0....,1.000000,1.000000,1.000000,0.0,optimal,0,0,1.000000,1.000000
3,gcnn:baseline,uniform5,3,small,data/instances/setcover/transfer_500r_1000c_0....,1.000000,1.000000,1.000000,0.0,optimal,0,0,1.000000,1.000000
4,gcnn:baseline,uniform5,4,small,data/instances/setcover/transfer_500r_1000c_0....,1.000000,1.000000,1.000000,0.0,optimal,0,0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,gcnn:baseline,depthK2,0,medium,data/instances/setcover/transfer_1000r_1000c_0...,0.979381,0.978355,1.016051,0.0,optimal,0,0,1.004675,1.004902
596,gcnn:baseline,depthK2,1,medium,data/instances/setcover/transfer_1000r_1000c_0...,1.079646,0.968000,1.012638,0.0,optimal,0,0,1.018315,1.019512
597,gcnn:baseline,depthK2,2,medium,data/instances/setcover/transfer_1000r_1000c_0...,0.765957,0.886861,0.949618,0.0,optimal,0,0,0.956372,0.946387
598,gcnn:baseline,depthK2,3,medium,data/instances/setcover/transfer_1000r_1000c_0...,0.904762,0.851351,0.971550,0.0,optimal,0,0,0.977005,0.967532


In [305]:
var_df = df.groupby(['type','sampling_strategy','instance']).std() / df.groupby(['type','sampling_strategy','instance']).mean()

In [306]:
var_df.groupby(['type','sampling_strategy']).mean()

seed    nnodes      nlps     stime       gap  \
type   sampling_strategy                                                     
medium depthK             0.790569  0.162235  0.145407  0.142906       NaN   
       depthK2            0.790569  0.162596  0.146062  0.152325       NaN   
       uniform5           0.790569  0.147752  0.141749  0.151262       NaN   
small  depthK             0.790569  0.233088  0.157669  0.146623  1.391291   
       depthK2            0.790569  0.232860  0.160797  0.147184  0.949817   
       uniform5           0.790569  0.221302  0.156418  0.151314  0.913410   

                          ndomchgs  ncutoffs  walltime  proctime  
type   sampling_strategy                                          
medium depthK                  NaN       NaN  0.142318  0.142244  
       depthK2                 NaN       NaN  0.151456  0.151476  
       uniform5                NaN       NaN  0.150407  0.150383  
small  depthK                  NaN       NaN  0.145234  0.145272  
       depthK2                 NaN       NaN  0.146318  0.146366  
       uniform5                NaN       NaN  0.150160  0.149993

In [189]:
ref_row = df[
        (df['sampling_strategy'] == 'uniform5') &
        (df['policy'] == df.iloc[i].policy) &
        (df['seed'] == df.iloc[i].seed) &
        (df['type'] == df.iloc[i].type) &
        (df['instance'] == df.iloc[i].instance)
    ]

In [151]:
df.at[0,'policy']

'gcnn:baseline'

In [102]:
df.iloc[0].stime /= 

5.15

In [103]:
i=0

In [104]:
df.iloc[i].policy

5.15

In [117]:
df.iloc[i].policy

'gcnn:baseline'

In [126]:
len(df[
    df['sampling_strategy'] == 'uniform5'
])

200

In [115]:
df.iloc[i].policy

'gcnn:baseline'

In [113]:
df['policy'] == df.iloc[i].policy

0      True
1      True
2      True
3      True
4      True
       ... 
595    True
596    True
597    True
598    True
599    True
Name: policy, Length: 600, dtype: bool

In [45]:
df.groupby(['type','sampling_strategy']).mean()

seed     nnodes       nlps      stime       gap  \
type   sampling_strategy                                                    
big    depthK              0.0  170007.00  215813.00  3600.0000  0.056156   
       uniform5            2.0  181774.20  210967.40  3600.0040  0.047812   
medium depthK              2.0   12181.90   14888.19   192.4859  0.000000   
       depthK2             2.0   10636.07   13019.47   170.2660  0.000000   
       uniform5            2.0   10177.52   12328.17   162.8540  0.000000   
small  depthK              2.0     313.82     560.60     7.6083  0.000000   
       depthK2             2.0     291.16     532.13     7.3189  0.000000   
       uniform5            2.0     285.20     524.75     7.2423  0.000000   

                          ndomchgs  ncutoffs     walltime     proctime  
type   sampling_strategy                                                
big    depthK                  0.0       0.0  3794.997987  3749.140625  
       uniform5                0.0       0.0  3796.600195  3751.600000  
medium depthK                  0.0       0.0   201.279468   198.852344  
       depthK2                 0.0       0.0   177.896081   175.830781  
       uniform5                0.0       0.0   170.154201   168.178125  
small  depthK                  0.0       0.0     7.911259     7.859531  
       depthK2                 0.0       0.0     7.615790     7.554531  
       uniform5                0.0       0.0     7.543738     7.471563

In [47]:
df.groupby(['type','sampling_strategy']).std()

seed        nnodes          nlps       stime  \
type   sampling_strategy                                                     
big    depthK                  NaN           NaN           NaN         NaN   
       uniform5           1.581139   5350.840607   4295.508910    0.011402   
medium depthK             1.421338  18637.800749  22703.497752  280.484343   
       depthK2            1.421338  16240.267802  20064.446911  246.617712   
       uniform5           1.421338  15210.605585  18292.163278  229.827537   
small  depthK             1.421338    319.033622    431.177013    4.471304   
       depthK2            1.421338    289.553914    402.072149    4.020209   
       uniform5           1.421338    282.772743    392.137097    3.949317   

                              gap  ndomchgs  ncutoffs    walltime    proctime  
type   sampling_strategy                                                       
big    depthK                 NaN       NaN       NaN         NaN         NaN  
       uniform5           0.00426       0.0       0.0    2.036686    1.554809  
medium depthK             0.00000       0.0       0.0  293.656144  290.006661  
       depthK2            0.00000       0.0       0.0  257.936460  254.808054  
       uniform5           0.00000       0.0       0.0  240.481379  237.458221  
small  depthK             0.00000       0.0       0.0    4.692819    4.643507  
       depthK2            0.00000       0.0       0.0    4.241291    4.183581  
       uniform5           0.00000       0.0       0.0    4.156231    4.107342

In [65]:
df

,policy,sampling_strategy,seed,type,instance,nnodes,nlps,stime,gap,status,ndomchgs,ncutoffs,walltime,proctime
0,gcnn:baseline,uniform5,0,small,data/instances/setcover/transfer_500r_1000c_0....,107,253,5.15,0.000000,optimal,0,0,5.440188,5.437500
1,gcnn:baseline,uniform5,1,small,data/instances/setcover/transfer_500r_1000c_0....,98,232,4.76,0.000000,optimal,0,0,4.913453,4.921875
2,gcnn:baseline,uniform5,2,small,data/instances/setcover/transfer_500r_1000c_0....,101,269,4.78,0.000000,optimal,0,0,4.939524,4.906250
3,gcnn:baseline,uniform5,3,small,data/instances/setcover/transfer_500r_1000c_0....,103,341,5.11,0.000000,optimal,0,0,5.234386,5.203125
4,gcnn:baseline,uniform5,4,small,data/instances/setcover/transfer_500r_1000c_0....,98,276,4.70,0.000000,optimal,0,0,4.942747,4.890625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,gcnn:baseline,uniform5,1,big,data/instances/setcover/transfer_2000r_1000c_0...,177157,206783,3600.00,0.055161,timelimit,0,0,3793.791117,3750.953125
602,gcnn:baseline,uniform5,2,big,data/instances/setcover/transfer_2000r_1000c_0...,183612,205886,3600.01,0.047833,timelimit,0,0,3798.593933,3753.296875
603,gcnn:baseline,uniform5,3,big,data/instances/setcover/transfer_2000r_1000c_0...,187736,214256,3600.00,0.044949,timelimit,0,0,3797.860517,3750.203125
604,gcnn:baseline,uniform5,4,big,data/instances/setcover/transfer_2000r_1000c_0...,175222,213008,3599.99,0.045884,timelimit,0,0,3797.610391,3753.250000
